In [1]:
using Plots;plotly()
using Optim
using Zygote
using Meteor.Utility
using Meteor.ExactDiag
using DataFrames;using CSV
using Pkg
using LinearAlgebra
ccall((:openblas_get_num_threads64_, Base.libblas_name),Cint,())
LinearAlgebra.BLAS.set_num_threads(8)
using Base.Threads;nthreads()

1

In [2]:
function thermal_state(dims,nbar)
    c::Array{Complex{Float64}}=zeros(dims,dims)
    if nbar==0
        c[1,1]=1
    else
        for i in 1:dims
           c[i,i]= exp(-i*log(1/nbar+1))
        end
    end
    c=c/tr(c)
    return c        
end

function fock_state(dims,n)
    c::Matrix{ComplexF64}=zeros(dims,dims)
    if n<dims
        c[n+1,n+1]=1
    end
    return c
end

function eye(n)
    k::Array{Complex{Float64}}=zeros(n,n)
    for i in 1:n
        k[i,i]=1
    end
    return k    
end


function a₋(n)      #n为维数
    s::Array{Complex{Float64}}=zeros(n,n)
    for i=1:n-1
        s[i,i+1]=sqrt(i)
    end
    return s
end

a₊(n)=adjoint(a₋(n));

dim=10
ν = 2 * pi * 1.3
initial_state=kron([1.0 0 0 0;0 0 0 0;0 0 0 0;0 0 0 0],thermal_state(dim,1))

function dm2vec(dm)#态向量化
#julia里是按列reshape的,reshape成列向量
    return reshape(dm,:,1)
end

function vec2dm(vec)
    return reshape(vec,4*dim,:)
end

#按照g,r,e,t的顺序
n=kron(eye(4),a₊(dim)*a₋(dim))
ee=kron([0 0 0 0;0 0 0 0;0 0 1 0;0 0 0 0],eye(dim))
rr=kron([0 0 0 0;0 1 0 0;0 0 0 0;0 0 0 0],eye(dim))
tt=kron([0 0 0 0;0 0 0 0;0 0 0 0;0 0 0 1],eye(dim))

η=0.15
costheta = sqrt(2)/2
γg = 2 * pi * 6.89 
γr = 2 * pi * 13.79  

a=a₊(dim)+a₋(dim)
ge=kron([0 0 1 0;0 0 0 0;0 0 0 0;0 0 0 0],exp(im*η*costheta*a))
eg=kron([0 0 0 0;0 0 0 0;1 0 0 0;0 0 0 0],exp(-im*η*costheta*a))
u=ge+eg
rt=kron([0 0 0 0;0 0 0 1;0 0 0 0;0 0 0 0],exp(im*η*costheta*a))
t_r=kron([0 0 0 0;0 0 0 0;0 0 0 0;0 1 0 0],exp(-im*η*costheta*a))
u2=rt+t_r
uu=u+u2
r_e=kron([0 0 0 0;0 0 1 0;0 0 0 0;0 0 0 0],exp(-im*η*costheta*a))
er=kron([0 0 0 0;0 0 0 0;0 1 0 0;0 0 0 0],exp(im*η*costheta*a))
u3=r_e+er

E=eye(4*dim)

function diss1()
    dtheta = 0.01
    x=zeros((4*dim)^2,(4*dim)^2)
    for theta in -1:dtheta:1
        c = dtheta * 0.75 * (1 + theta^2)
        jump = sqrt(c / 2) * exp(im*η*theta*a)
        aup=kron([0 0 0 0;0 0 0 0;1 0 0 0;0 0 0 0],jump')
        adown=kron([0 0 1 0;0 0 0 0;0 0 0 0;0 0 0 0],jump)
        x=x+kron(transpose(aup),adown)-0.5*kron(E,aup*adown)-0.5*kron(transpose(aup*adown),E)
    end
    return x
end
function diss2()
    dtheta = 0.01
    x=zeros((4*dim)^2,(4*dim)^2)
    for theta in -1:dtheta:1
        c = dtheta * 0.75 * (1 + theta^2)
        jump = sqrt(c / 2) * exp(im*η*theta*a)
        aup=kron([0 0 0 0;0 0 0 0;0 0 0 0;0 1 0 0],jump')
        adown=kron([0 0 0 0;0 0 0 1;0 0 0 0;0 0 0 0],jump)
        x=x+kron(transpose(aup),adown)-0.5*kron(E,aup*adown)-0.5*kron(transpose(aup*adown),E)
    end
    return x
end
function diss3()
    dtheta = 0.01
    x=zeros((4*dim)^2,(4*dim)^2)
    for theta in -1:dtheta:1
        c = dtheta * 1.5 * (1 - theta^2)
        jump = sqrt(c / 2) * exp(im*η*theta*a)
        aup=kron([0 0 0 0;0 0 0 0;0 1 0 0;0 0 0 0],jump')
        adown=kron([0 0 0 0;0 0 1 0;0 0 0 0;0 0 0 0],jump)
        x=x+kron(transpose(aup),adown)-0.5*kron(E,aup*adown)-0.5*kron(transpose(aup*adown),E)
    end
    return x
end
function diss4()
    dtheta = 0.01
    x=zeros((4*dim)^2,(4*dim)^2)
    for theta in -1:dtheta:1
        c = dtheta * 1.5 * (1 - theta^2)
        jump = sqrt(c / 2) * exp(im*η*theta*a)
        aup=kron([0 0 0 0;0 0 0 0;0 0 0 0;1 0 0 0],jump')
        adown=kron([0 0 0 1;0 0 0 0;0 0 0 0;0 0 0 0],jump)
        x=x+kron(transpose(aup),adown)-0.5*kron(E,aup*adown)-0.5*kron(transpose(aup*adown),E)
    end
    return x
end
dissipation1=γg*diss1()
dissipation2=γr*diss2()
dissipation3=γr*diss3()
dissipation4=γg*diss4()
instate=dm2vec(initial_state);

In [3]:
function eit_Lindbladian_vec(ΩL1,ΩL2,Δg,Δr)
    unitary::Matrix{ComplexF64}=kron(E,-im*(ν*n-Δg*ee+(Δr-Δg)*rr+(Δr-Δg-(Δg+8*ν))*tt+ΩL1/2*uu+ΩL2/2*u3))+kron(transpose(im*(ν*n-Δg*ee+(Δr-Δg)*rr+(Δr-Δg-(Δg+8*ν))*tt+ΩL1/2*uu+ΩL2/2*u3)),E)
    
    return unitary+dissipation1+dissipation2+dissipation3+dissipation4
end

eit_Lindbladian_vec (generic function with 1 method)

In [12]:
eit_vec=eit_Lindbladian_vec(2 * pi * 2.236, 2 * pi * 23.7 ,2 * pi * 108, 2 * pi * 108)
list=[real(tr(n*vec2dm(instate)))]
delta_t=5
f=instate
evo=exp(delta_t*eit_vec)
@time for i in 1:100
    f=evo*f
    print(i," ")
    append!(list,real(tr(n*vec2dm(f)))) 
end
plot(0:5:500,list)

1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100   0.213522 seconds (4.17 k allocations: 5.005 MiB)


<!DOCTYPE html>
 
 
 Plots.jl

In [13]:
function feval_40(x)
    f=exp(40*eit_Lindbladian_vec(x[1],x[2],x[3],x[4]))*instate
    a=real(tr(n*vec2dm(f)))
    print(a," ")
    return a
end
function feval_90(x)
    f=exp(90*eit_Lindbladian_vec(x[1],x[2],x[3],x[4]))*instate
    a=real(tr(n*vec2dm(f)))
    print(a," ")
    return a
end
function feval_150(x)
    f=exp(150*eit_Lindbladian_vec(x[1],x[2],x[3],x[4]))*instate
    a=real(tr(n*vec2dm(f)))
    print(a," ")
    return a
end
function feval_250(x)
    f=exp(250*eit_Lindbladian_vec(x[1],x[2],x[3],x[4]))*instate
    a=real(tr(n*vec2dm(f)))
    print(a," ")
    return a
end

function g_40!(G,seq)
    print("g!  ")
    a=real(gradient(feval_40,seq)[1])
    for i in 1:length(seq)
       G[i]=a[i] 
    end
end
function g_90!(G,seq)
    print("g!  ")
    a=real(gradient(feval_90,seq)[1])
    for i in 1:length(seq)
       G[i]=a[i] 
    end
end
function g_150!(G,seq)
    print("g!  ")
    a=real(gradient(feval_150,seq)[1])
    for i in 1:length(seq)
       G[i]=a[i] 
    end
end
function g_250!(G,seq)
    print("g!  ")
    a=real(gradient(feval_250,seq)[1])
    for i in 1:length(seq)
       G[i]=a[i] 
    end
end;

In [15]:
#初始
# ΩL1 = 2 * pi * 2.236 
# ΩL2 = 2 * pi * 23.7 
# Δg = 2 * pi * 108 
# Δr = 2 * pi * 108 
# Δt = 2 * pi * 117.33
initial_x=[2 * pi * 2.236, 2 * pi * 23.7 ,2 * pi * 108, 2 * pi * 108 ]
# res_40=optimize(feval_40,g_40!,initial_x,LBFGS())
# res_90=optimize(feval_90,g_90!,initial_x,LBFGS())
# res_150=optimize(feval_150,g_150!,initial_x,LBFGS())
# res_250=optimize(feval_250,g_250!,initial_x,LBFGS())

4-element Array{Float64,1}:
  14.049202346853557
 148.91149178015618
 678.5840131753953
 678.5840131753953

In [16]:
# initial：
# 14.049202346853557, 148.91149178015618, 678.5840131753953, 678.5840131753953

# 40:
#  29.414155980164807, 143.02321686599555, 562.7272532543809, 561.1661331065062

# 90:
#  22.22895126792895, 182.74814937263437, 921.73389263177, 920.6575268329686

# 150:
#  18.39374569668173, 218.65695647321104, 1325.0343585188957, 1324.110515444204

# 250:
#  15.061759135858427, 265.08526852940776, 1953.720834601189, 1952.876854452561

In [4]:
function evolist(x)
    eit_vec=eit_Lindbladian_vec(x[1],x[2],x[3],x[4])
    list=[real(tr(n*vec2dm(instate)))]
    delta_t=5
    f=instate
    evo=exp(delta_t*eit_vec)
    @time for i in 1:100
        f=evo*f
        print(i," ")
        append!(list,real(tr(n*vec2dm(f)))) 
    end  
    return list
end
# list0=evolist([2 * pi * 2.236, 2 * pi * 23.7 ,2 * pi * 108, 2 * pi * 108])
list0=evolist([18.99157064667333 201.88167588498618 1248.3702369513921 1248.3702369513921])
list_40=evolist([29.414155980164807, 143.02321686599555, 562.7272532543809, 561.1661331065062])
list_90=evolist([22.22895126792895, 182.74814937263437, 921.73389263177, 920.6575268329686])
list_150=evolist([18.39374569668173, 218.65695647321104, 1325.0343585188957, 1324.110515444204])
list_250=evolist([15.061759135858427, 265.08526852940776, 1953.720834601189, 1952.876854452561])

1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100   0.266540 seconds (158.97 k allocations: 13.406 MiB)
1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100   0.215045 seconds (4.28 k allocations: 5.015 MiB)
1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100   0.197746 seconds 

101-element Array{Float64,1}:
 0.9902248289345064
 0.8561737391337629
 0.7089056026615133
 0.6076057428289674
 0.5222705134058558
 0.44777716994727734
 0.38332628007096825
 0.32761260340750664
 0.27963561674438586
 0.2383567882229135
 0.20296021462510325
 0.17262709379151975
 0.146706102477234
 ⋮
 0.0008256446716496523
 0.0008256185759164352
 0.0008255968546688492
 0.0008255787771842976
 0.0008255637311309057
 0.0008255512098576201
 0.0008255407889623153
 0.000825532117227462
 0.0008255249005574345
 0.0008255188955291029
 0.0008255138983787482
 0.0008255097404227595

In [5]:
plot(0:5:500,list0,label="ref")
plot!(0:5:500,list_40,label="40")
plot!(0:5:500,list_90,label="90")
plot!(0:5:500,list_150,label="150")
plot!(0:5:500,list_250,label="250")

<!DOCTYPE html>
 
 
 Plots.jl

In [6]:
df=DataFrame(list_T=0:5:500,list_ZERO=list0,list_40=list_40,list_90=list_90,list_150=list_150,list_250=list_250);CSV.write("eit_four_opt2.csv", df);